In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Spectral7
from bokeh.io import output_notebook
#from graphviz import Digraph
import numpy as np
import cnnUtils
import time
import copy
import os
import PIL
%matplotlib inline
plt.ion()
output_notebook()

Please install sklearn for layer visualization


Loading BokehJS ...

In [5]:
baseDirectory = 'g:/Selim/Thesis/Code/'
setDirectory = 'EBA'
setImageSize = 128

# EBA5 cropped mean and std values
#0.544978628454
#0.0564096715989
setMean = [0.544, 0.544, 0.544]
setStd = [0.056, 0.056, 0.056]

# EBA5 full mean and std
#Dim 0 mean: 0.558987606566
#Dim 0 stdv: 0.0675306702862
#setMean = [0.558, 0.558, 0.558]
#setStd = [0.067, 0.067, 0.067]

dataTransforms = {
    'train': transforms.Compose([
        transforms.Scale(setImageSize),
        transforms.RandomCrop(setImageSize),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=setMean, std=setStd)
    ]),
    'test': transforms.Compose([
        transforms.Scale(setImageSize),
        transforms.CenterCrop(setImageSize),
        transforms.ToTensor(),
        transforms.Normalize(mean=setMean, std=setStd)
    ]),
}

setPath = os.path.join(baseDirectory, setDirectory)
datasets = {x: torchvision.datasets.ImageFolder(os.path.join(setPath, x), dataTransforms[x])
           for x in ['train', 'test']}

datasetLoaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=10, shuffle=True, num_workers=4)
                for x in ['train', 'test']}

testLoader = torch.utils.data.DataLoader(datasets['test'], batch_size=1, shuffle=False, num_workers=4)

datasetSizes = {x: len(datasets[x]) for x in ['train', 'test']}
datasetClasses = datasets['train'].classes

useGPU = torch.cuda.is_available()

print(str(datasetSizes) + ' images will be used.' )
print('GPU will ' + ('' if useGPU else 'not ') + 'be used.' )
print(str(len(datasetClasses)) + ' output classes')

{'train': 607, 'test': 311} images will be used.
GPU will be used.
2 output classes


In [33]:
class Autoencoder(nn.Module):
    def __init__(self, ):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(49152, 32)
        self.decoder = nn.Linear(16, 49152)
        self.sigmoid = nn.Sigmoid()
    
    def reparametrize(self, mu, log_var):
        """"z = mean + eps * sigma where eps is sampled from N(0, 1)."""
        eps = Variable(torch.randn(mu.size(0), mu.size(1))).cuda()
        z = mu + eps * torch.exp(log_var/2)    # 2 for convert var to std
        return z

    def forward(self, x):
        h = self.encoder(x)
        mu, log_var = torch.chunk(h, 2, dim=1)  # mean and log variance.
        z = self.reparametrize(mu, log_var)
        print(z)
        out = self.decoder(z)
        return out, mu, log_var

In [34]:
ae = Autoencoder()

if torch.cuda.is_available():
    ae.cuda()

In [35]:
optimizer = torch.optim.Adam(ae.parameters(), lr=0.001)
iter_per_epoch = len(datasetLoaders['train'])

data_iter = iter(datasetLoaders['train'])

# fixed inputs for debugging
fixed_z = Variable(torch.randn(100, 20)).cuda()
fixed_x, _ = next(data_iter)
#torchvision.utils.save_image(fixed_x.cpu(), './data/real_images.png')
fixed_x = Variable(fixed_x.view(fixed_x.size(0), -1)).cuda()
print(fixed_x)

Variable containing:
-0.8207 -0.9608 -0.1204  ...   0.1597  0.2297 -0.1204
-0.8908 -0.8908 -0.8908  ...  -0.8908 -0.8908 -0.8908
-1.9412 -1.6611 -1.6611  ...   0.0196 -0.4706 -1.1709
          ...             ⋱             ...          
-1.0308 -1.4510 -0.9608  ...  -0.1905 -0.0504 -0.0504
-3.4118 -3.2717 -2.9916  ...  -1.3109 -1.3810 -1.4510
 8.1429  8.1429  8.1429  ...   8.1429  8.1429  8.1429
[torch.cuda.FloatTensor of size 10x49152 (GPU 0)]



In [36]:
for epoch in range(10):
    for i, (images, _) in enumerate(datasetLoaders['train']):
        
        images = Variable(images.view(images.size(0), -1)).cuda()
        out, mu, log_var = ae(images)
        
        # Compute reconstruction loss and kl divergence
        # For kl_divergence, see Appendix B in the paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(out, images, size_average=False)
        kl_divergence = torch.sum(0.5 * (mu**2 + torch.exp(log_var) - log_var -1))
        
        # Backprop + Optimize
        total_loss = reconst_loss + kl_divergence
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print ("Epoch[%d/%d], Step [%d/%d], Total Loss: %.4f, "
                   "Reconst Loss: %.4f, KL Div: %.7f" 
                   %(epoch+1, 50, i+1, iter_per_epoch, total_loss.data[0], 
                     reconst_loss.data[0], kl_divergence.data[0]))
    
    # Save the reconstructed images
    reconst_images, _, _ = ae(fixed_x)
    reconst_images = reconst_images.view(reconst_images.size(0), 1, 28, 28)
    cnnUtils.ImShow(reconst_images.data.cpu())
    #torchvision.utils.save_image(reconst_images.data.cpu(), 
    #    './data/reconst_images_%d.png' %(epoch+1))

Variable containing:

Columns 0 to 9 
 0.2197  0.1477 -0.7281  0.0795 -0.4090  0.7197 -1.7887 -0.7594  0.3859  0.2296
-1.0565 -0.1180 -0.2778 -1.7931 -0.5626  1.8512  1.3527 -0.2260  0.3265 -1.0619
 0.0164 -1.8499  0.4539  0.3770  0.2654  0.3573 -1.1490  1.0930  1.0683  0.6408
-1.0967  1.6292 -1.6615 -0.3203 -1.3576  0.5268  0.4195  0.7479 -0.6778 -2.5452
 0.6148  1.8944 -0.6065 -0.8592 -0.8432  1.1121  1.1463  1.1980 -2.1773  1.6864
-0.0470  1.2872 -1.4842  0.5642  0.3268 -0.3462 -0.5390  1.3328 -1.0188 -0.9640
-2.5094  1.0995 -0.2575  0.0653  0.3099  0.9514  1.5063 -4.1697  0.6379  0.0394
-3.2450  0.3512 -0.3461  0.8071  1.1521 -0.4025  0.7949  0.2796 -2.1194 -0.9994
-1.1069 -0.8133  0.9961  1.5692  0.2963 -1.6386  0.0100  0.2086 -1.0161  0.1804
 0.2309  0.9537 -2.5820 -0.8169  0.1842  0.7425  2.4493 -1.8214 -1.9311  0.2252

Columns 10 to 15 
-1.0360  1.0725  0.5995  0.9709 -2.5891 -0.6164
-0.6308  1.7817 -0.0434  1.4297 -0.5741 -0.8999
 0.5182  0.0053  0.9393  2.0472 -0.4046  0.2051

Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 15 
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan



Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 15 
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan

Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 15 
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan


Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 15 
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan


Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 15 
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan
  nan   nan   nan


RuntimeError: size '[10 x 1 x 28 x 28]' is invalid for input of with 491520 elements at D:\Downloads\pytorch-master-1\torch\lib\TH\THStorage.c:59